In [0]:
import tensorflow

In [0]:
!pip install -U tensorflow_datasets
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [0]:
!pip install -U tensorflow_hub

In [0]:
%history


In [0]:
import time #we will be using time stamp to name the modules
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
#tfds.disable_progress_bar()

from tensorflow.keras import layers 

In [0]:
(train_examples , validation_examples) , info =tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]','train[80%:]'],#slicing with 80% for train and rest for validation
    with_info=True,
    as_supervised=True
)

In [0]:
for i , example in enumerate(validation_examples.take(5)):
  print(example[0] , example[1] , i)  
#they are in various shapes

In [0]:
def format_image(image , label):
  #gonna use MobileNet pretrained model so it has an input resolution of 224 and also normalizing
  image = tf.image.resize(image , (IMAGE_RES , IMAGE_RES))/255.0
  return image , label 
num_examples = info.splits['train'].num_examples
BATCH_SIZE = 32
IMAGE_RES = 224 
train_batches= train_examples.cache().shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
#prefetch helps us to make ready the next set of examples
validation_batches = validation_examples.cache().map(format_image).batch(BATCH_SIZE).prefetch(1)

In [0]:
print(train_examples)

In [0]:
URL="https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor=hub.KerasLayer(
    URL , trainable=False , input_shape=(IMAGE_RES , IMAGE_RES , 3)
)

In [0]:
model = tf.keras.Sequential([
  feature_extractor,
  layers.Dense(2)
])

model.summary()

In [0]:
model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy']
)

EPOCHS = 1
history = model.fit(train_batches,
                    epochs = EPOCHS , 
                    validation_data = validation_batches)

In [0]:
image_batch, label_batch = next(iter(train_batches.take(1)))
image_batch = image_batch.numpy()
label_batch = label_batch.numpy()

predicted_batch = model.predict(image_batch)
predicted_batch = tf.squeeze(predicted_batch).numpy()
predicted_ids = np.argmax(predicted_batch, axis=-1)
predicted_class_names = class_names[predicted_ids]
predicted_class_names

In [0]:
print("Labels: ", label_batch)
print("Predicted labels: ", predicted_ids)

In [0]:
plt.figure(figsize=(10,9))
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  color = "blue" if predicted_ids[n] == label_batch[n] else "red"
  plt.title(predicted_class_names[n].title(), color=color)
  plt.axis('off')
_ = plt.suptitle("Model predictions (blue: correct, red: incorrect)")

In [0]:
t = time.time()
export_path_keras = "./{}.h5".format(int(t))
print(export_path_keras)
model.save(export_path_keras)

In [0]:
!ls

In [0]:
reloaded = tf.keras.models.load_model(
    export_path_keras,
    #`custom_objects` tells keras how to load a `hub.KerasLayer`
    custom_objects = {'KerasLayer':hub.KerasLayer}
)
reloaded.summary()

In [0]:
result=model.predict(image_batch)
reloaded_result_batch = reloaded.predict(image_batch)

In [0]:
EPOCHS = 3
history = reloaded.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)

In [0]:
t = time.time()

export_path_sm = "./{}".format(int(t))
print(export_path_sm)

tf.saved_model.save(model, export_path_sm)

In [0]:
!ls {export_path_sm}

In [0]:
reloaded_sm = tf.saved_model.load(export_path_sm)

In [0]:
reload_sm_result_batch = reloaded_sm(image_batch, training=False).numpy()

In [0]:
t = time.time()

export_path_sm = "./{}".format(int(t))
print(export_path_sm)
tf.saved_model.save(model, export_path_sm)

In [0]:
reload_sm_keras = tf.keras.models.load_model(
  export_path_sm,
  custom_objects={'KerasLayer': hub.KerasLayer})

reload_sm_keras.summary()

In [0]:
result_batch = model.predict(image_batch)
reload_sm_keras_result_batch = reload_sm_keras.predict(image_batch)

In [0]:
!zip -r model.zip {export_path_sm}

In [0]:
!ls